In [7]:
import Functions as Func
import AdvancedFuncs as af


import xgboost as xgb
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error
from xgboost.callback import EarlyStopping

import optuna

Here we outline the process to perform our XGBoost hyperparameter tuning

The main motivation for our approach is the difficulty tuning XGBoost using cross validation, due to its ability to quickly overfit to validation data that it is allowed to iterate on. This required a method which did not allow the model to keep using the same dataset repeatedly, so it could not infer its properties and overfit to it as highly flexible models are capable of.

XGBoost also has a large number of sensitive parameters to tune, which motivated our decision to use a mixed random grid search-Bayesian optimization approach which we implemented with Optuna. This approach is very quick to find effective hyperparameter candidates, but it can also make overfitting worse if the objective is set improperly.

We avoided the overfitting problem by using a boostrapping approach, where both training data and validation data are selected from boostrapped samples, for each iteration. This means that our model never seems the same dataset more than once, which largely eliminates the problem of overfitting as the model is incentivized to learn the overall trends of the data rather than fit to noise.

In [8]:
# We first go through our normal data pre-processing steps

df = Func.load_and_basic_process()
df_X, df_Y = Func.df_data_process(df)

X_model, X_test, Y_model, Y_test = train_test_split(df_X, df_Y, test_size=0.1, random_state=1)
X_train, X_val, Y_train, Y_val = train_test_split(X_model, Y_model, test_size=0.2, random_state=2)

In [16]:
def objective(trial):
    
    # We define a hyperparameter space to optimize. We could expand this to additional parameters or expand the range of existing ones, if desired

    n = trial.suggest_int('n', 400, 900)
    params = {
        'objective': 'reg:absoluteerror',
        'eval_metric': 'mae',
        "tree_method": "hist",
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'max_delta_step': trial.suggest_float('max_delta_step', 0, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 0, 10),
        'gamma': trial.suggest_float('gamma', 0, 5)
    }
    
    # We define boostrap training/validation sets from the original model data for each loop
    X_Train_boot, Y_Train_boot, X_Val_boot, Y_Val_boot = af.generate_train_val_sets(X_model, Y_model)
    
    # We convert the boostrapped dataframes into efficient XGBoost DMatrix format
    dtrain_reg = xgb.DMatrix(X_Train_boot, label=Y_Train_boot["Frequency"], weight=Y_Train_boot["Exposure"])
    dval_reg = xgb.DMatrix(X_Val_boot, label=Y_Val_boot["Frequency"], weight=Y_Val_boot["Exposure"])
    

    evals = [(dtrain_reg, 'train'), (dval_reg, 'eval')]

    # Here we train our model on our current set of hyperparameters
    early_stopping_cb = EarlyStopping(rounds=10, save_best=True)
    model = xgb.train(
        params=params,
        dtrain=dtrain_reg,
        num_boost_round=n,
        evals=evals,
        callbacks=[early_stopping_cb],
    )

    # Generate predictions and calculate MAE
    best_iteration = model.best_iteration
    preds = model.predict(dval_reg, iteration_range=(0, best_iteration + 1))
    mae = mean_absolute_error(Y_Val_boot["Frequency"], preds, sample_weight=Y_Val_boot["Exposure"])

    # Our optimizer will try to minimize the MAE returned 
    return mae

In [17]:
# We initialize our experiment here and set the number of trials
# Our output is verbose so that we can see the stability of the XGBoost within its internal validation process on training data in addition to our own validation process

trial_num = 50

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=trial_num)  

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-02-24 20:01:14,512] A new study created in memory with name: no-name-4f06a18a-e38b-4201-bdc4-4f8b06582853


[0]	train-mae:0.07383	eval-mae:0.07310
[1]	train-mae:0.07383	eval-mae:0.07310
[2]	train-mae:0.07383	eval-mae:0.07310
[3]	train-mae:0.07383	eval-mae:0.07310
[4]	train-mae:0.07383	eval-mae:0.07310
[5]	train-mae:0.07383	eval-mae:0.07310
[6]	train-mae:0.07383	eval-mae:0.07310
[7]	train-mae:0.07383	eval-mae:0.07310
[8]	train-mae:0.07383	eval-mae:0.07310
[9]	train-mae:0.07383	eval-mae:0.07310


[I 2024-02-24 20:01:17,649] Trial 0 finished with value: 0.07309952700929674 and parameters: {'n': 561, 'learning_rate': 0.10266542490882356, 'max_depth': 6, 'subsample': 0.8956579599377384, 'colsample_bytree': 0.6027697410312449, 'max_delta_step': 1.5380039113067434, 'min_child_weight': 2.7839419211058356, 'gamma': 4.6068297261904965}. Best is trial 0 with value: 0.07309952700929674.


[0]	train-mae:0.07366	eval-mae:0.07396
[1]	train-mae:0.07366	eval-mae:0.07396
[2]	train-mae:0.07366	eval-mae:0.07396
[3]	train-mae:0.07366	eval-mae:0.07396
[4]	train-mae:0.07366	eval-mae:0.07396
[5]	train-mae:0.07366	eval-mae:0.07396
[6]	train-mae:0.07366	eval-mae:0.07396
[7]	train-mae:0.07366	eval-mae:0.07396
[8]	train-mae:0.07366	eval-mae:0.07396
[9]	train-mae:0.07366	eval-mae:0.07396
[10]	train-mae:0.07366	eval-mae:0.07396


[I 2024-02-24 20:01:20,286] Trial 1 finished with value: 0.07396152844599894 and parameters: {'n': 635, 'learning_rate': 0.021690651840594984, 'max_depth': 1, 'subsample': 0.6167348513129736, 'colsample_bytree': 0.9094570674399696, 'max_delta_step': 6.264494440190244, 'min_child_weight': 9.277285005283044, 'gamma': 4.85806611249021}. Best is trial 0 with value: 0.07309952700929674.


...

[0]	train-mae:0.07377	eval-mae:0.07371
[1]	train-mae:0.07377	eval-mae:0.07371
[2]	train-mae:0.07377	eval-mae:0.07371
[3]	train-mae:0.07377	eval-mae:0.07371
[4]	train-mae:0.07377	eval-mae:0.07371
[5]	train-mae:0.07377	eval-mae:0.07371
[6]	train-mae:0.07377	eval-mae:0.07371
[7]	train-mae:0.07377	eval-mae:0.07371
[8]	train-mae:0.07377	eval-mae:0.07371
[9]	train-mae:0.07377	eval-mae:0.07371
[10]	train-mae:0.07377	eval-mae:0.07371


[I 2024-02-24 20:03:52,547] Trial 48 finished with value: 0.07370626330594769 and parameters: {'n': 672, 'learning_rate': 0.1673246046754787, 'max_depth': 1, 'subsample': 0.8049276336306977, 'colsample_bytree': 0.7451148639687023, 'max_delta_step': 1.9971729733169732, 'min_child_weight': 0.8796864159457334, 'gamma': 2.105150056301967}. Best is trial 35 with value: 0.07225031507802897.


[0]	train-mae:0.07333	eval-mae:0.07373
[1]	train-mae:0.07333	eval-mae:0.07373
[2]	train-mae:0.07333	eval-mae:0.07373
[3]	train-mae:0.07333	eval-mae:0.07373
[4]	train-mae:0.07333	eval-mae:0.07373
[5]	train-mae:0.07333	eval-mae:0.07373
[6]	train-mae:0.07333	eval-mae:0.07373
[7]	train-mae:0.07333	eval-mae:0.07373
[8]	train-mae:0.07333	eval-mae:0.07373
[9]	train-mae:0.07333	eval-mae:0.07373


[I 2024-02-24 20:03:55,774] Trial 49 finished with value: 0.07373045728503656 and parameters: {'n': 505, 'learning_rate': 0.08607804756718676, 'max_depth': 3, 'subsample': 0.8683794931449266, 'colsample_bytree': 0.544007513104586, 'max_delta_step': 3.8232426806443565, 'min_child_weight': 7.119317552533431, 'gamma': 1.794858764405061}. Best is trial 35 with value: 0.07225031507802897.


Number of finished trials:  50
Best trial:
  Value:  0.07225031507802897
  Params: 
    n: 597
    learning_rate: 0.07551198765186194
    max_depth: 1
    subsample: 0.5658703466710434
    colsample_bytree: 0.8832081728582752
    max_delta_step: 0.36546191800351824
    min_child_weight: 8.590314483783583
    gamma: 2.798075767159713
